In [15]:
import sys 

'''
ignore __pycache__
add path to repo
'''
sys.dont_write_bytecode = True

import os
import pandas as pd
import numpy as np
from datetime import timedelta
import panel as pn
pn.extension('tabulator', sizing_mode="stretch_width")
import hvplot.pandas

import warnings
warnings.filterwarnings('ignore')

path = 'D:\Repos\PirnaCaseStudy'
sys.path.append(path)
import SMARTControl as sc
import git

import streamlit as st
from streamlit_folium import st_folium as stf

In [7]:
os.chdir(path)
database_fn = 'Data/Database.db'
Get = sc.queries.Get(database_fn)

In [8]:
'''
Querying the database
'''

MonitoringPointData_df = Get.MonitoringPointData(GageData = 1)
GageData_df = Get.GageData

# First and last date
start, end = Get.StartEndDate ()

#All variables 
Variables_df = Get.Table('Variables')

#Hydraulic heads
Get.LongTimeSeries(0)
df = Get.LongTimeSeries_df.copy()
df = df.set_index('Date')

#River data
Get.ShortTimeSeries(7, 'RG')
r_df = Get.ShortTimeSeries_df.copy()
r_df = r_df.set_index('Date')

In [12]:
map_gdf, river_gage_gdf = sc.utils.prepare_query (Get, '2022-12-12')


grid_x_gcs , grid_y_gcs , grid_z_gcs, U , V = sc.utils.Interpolation_Gradient (map_gdf , crs_utm = 25833 ,
                                                                                pixel_size = 10)

x = grid_x_gcs
y = grid_y_gcs

arrows_df = sc.utils.arrow_head (grid_x_gcs , grid_y_gcs , grid_z_gcs, U , V , scale = 10)
df = arrows_df [ arrows_df.index.isin( np.arange(0 ,
                                                  arrows_df.shape[0],
                                                  2
                                                )
                                      )
                ].reset_index (drop = True)



Map = sc.utils.Folium_map(Get)

Map_contour = sc.utils.Folium_contour ( 

    Map , map_gdf , river_gage_gdf , 
    grid_x_gcs , grid_y_gcs , grid_z_gcs

                                      )

# arrows_df
Map = sc.utils.Folium_arrows(Map_contour , df )

In [27]:
st.title('test')
m = stf(Map, width = 300)

{'last_clicked': None,
 'last_object_clicked': None,
 'all_drawings': None,
 'last_active_drawing': None,
 'bounds': {'_southWest': {'lat': 50.96458617, 'lng': 13.92186},
  '_northEast': {'lat': 50.96652906, 'lng': 13.92975382}},
 'zoom': 17,
 'last_circle_radius': None,
 'last_circle_polygon': None}